### APS 1052 Final Project

The focus of the team's model is going to be the seed model from the book Machine Learning and Data Science Blueprints for Finance. The model is going to be using stock data provided via the Google Drive, sub-folder > IntraDayData > TradeAndQuoteETFData. The goal of our model is to use the stock regression model provided from the book to predict the future price of an ETF. 


For this case study, the independent variables used are the following potentially correlated assets:

Stocks: Apple, Meta, Microsoft, Tesla

ETFs: QQQ, SPY

Indices: S&P500, Dow Jones, VIX

In [37]:
# Load libraries
import numpy as np
import pandas as pd

## Note -  Nov 25 meeting:
Features: SMA, RSI,ROC,MOM,lags

Technical Indicators: Talib (metioned above) VIX, historical(https://www.cboe.com/tradable_products/vix/vix_historical_data/), Alternative Money Flow, Put-Call Parity

Input processing: smoothing(exp)

Metrics: same as homework



In [38]:
import sklearn
import PIL

In [2]:
# Load libraries
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from matplotlib import pyplot
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor

#Libraries for Deep Learning Models
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from keras.layers import LSTM
from keras.wrappers.scikit_learn import KerasRegressor

#Libraries for Statistical Models
import statsmodels.api as sm

#Libraries for Saving the Model
from pickle import dump
from pickle import load

# Time series Models
from statsmodels.tsa.arima_model import ARIMA
#from statsmodels.tsa.statespace.sarimax import SARIMAX

# Error Metrics
from sklearn.metrics import mean_squared_error

# Feature Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression


#Plotting 
from pandas.plotting import scatter_matrix
from statsmodels.graphics.tsaplots import plot_acf


In [62]:
apple_data = pd.read_csv('AAPL.csv')
fb_data = pd.read_csv('FB.csv')
msft_data = pd.read_csv('MSFT.csv')
spy_data = pd.read_csv('SPY.csv')
tsla_data = pd.read_csv('TSLA.csv')
qqq_data = pd.read_csv('QQQ.csv')
vix_data = pd.read_csv('^VIX.csv')
djia_data = pd.read_csv('DJIA.csv')
sp500_data = pd.read_csv('SP500.csv')


In [17]:
import talib as ta
stk_tickers = ['AAPL', 'FB', 'MSFT','TSLA']
etf_tickers = ['SPY', 'QQQ']
idx_tickers = ['SP500', 'DJIA', 'VIX']

In [75]:
macd, macdsignal, macdhist = ta.MACD(tsla_data.Close.values, fastperiod=12, slowperiod=26, signalperiod=9)

In [42]:
def get_indicator(df,indicator):
    if 'MACD' in indicator: 
        macd, macdsignal, macdhist = ta.MACD(df.Close.values, fastperiod=12, slowperiod=26, signalperiod=9)
    if 'MFI' in indicator: 
        mfi = ta.MFI(np.reshape(tsla_data.High.values.astype('float64'),-1), \
            np.reshape(tsla_data.Low.values.astype('float64'),-1), np.reshape(tsla_data.Close.values.astype('float64'),-1),\
                 np.reshape(tsla_data.Volume.values.astype('float64'),-1), timeperiod=14)
    if 'MFI' in indicator: 
        mfi = ta.MFI(float(df.High.values), float(df.Low.values), float(df.Close.values), float(df.Volume.values), timeperiod=14)
    
    return mfi, macd, macdsignal, macdhist

In [ ]:
def get_indicator(df, indicator):
        ret_df = df
        if 'MACD' in indicator:
            macd, macdsignal, macdhist = ta.MACD(df.close.values, fastperiod=12, slowperiod=26, signalperiod=9)
            ret_df = KlineData._merge_dataframe(pd.DataFrame([macd, macdsignal, macdhist]).T.rename(columns={0: "macddif", 1: "macddem", 2: "macdhist"}), ret_df)
            ret_df = KlineData._merge_dataframe(line_intersections(ret_df, columns=['macddif', 'macddem']), ret_df)
        if 'MFI' in indicator:
            real = ta.MFI(df.high.values, df.low.values, df.close.values, df.volume.values, timeperiod=14)
            ret_df = KlineData._merge_dataframe(pd.DataFrame([real]).T.rename(columns={0: "mfi"}), ret_df)
        if 'ATR' in indicator:
            real = ta.NATR(df.high.values, df.low.values, df.close.values, timeperiod=14)
            ret_df = KlineData._merge_dataframe(pd.DataFrame([real]).T.rename(columns={0: "atr"}), ret_df)
        if 'ROCR' in indicator:
            real = ta.ROCR(df.close.values, timeperiod=10)
            ret_df = KlineData._merge_dataframe(pd.DataFrame([real]).T.rename(columns={0: "rocr"}), ret_df)
        ret_df['date'] = pd.to_datetime(ret_df['date'], format='%Y-%m-%d')
        return ret_df